In [23]:
!pip install pymupdf pytesseract pillow
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io

def extract_text_with_ocr(page) -> str:
    pix = page.get_pixmap(dpi=300)
    image = Image.open(io.BytesIO(pix.tobytes("png")))
    text = pytesseract.image_to_string(image, lang="eng")
    return text.strip()

def extract_text_from_pdf(path: str) -> str:
    doc = fitz.open(path)
    all_text = []

    for page in doc:
        text = page.get_text()
        if text.strip():
            all_text.append(text)
        else:
            ocr_text = extract_text_with_ocr(page)
            all_text.append(ocr_text)

    return "\n".join(all_text).strip()


In [24]:
cv_text = extract_text_from_pdf("/home/yaya/rhagent/data/cvs/Cv_Yahya_KROMBI.pdf")
offer_text = extract_text_from_pdf("/home/yaya/rhagent/data/jobs/offre2.pdf")

print("CV:\n", cv_text[:500])
print("\nOFFER:\n", offer_text[:500])


CV:
 KROMBI YAHYA
Dynamic and rigorous engineer seeking a position in the ﬁeld
of Data/Gen IA.
\ faM apMarker : 136 Boulev ard Maxime Gorki, Villejuif
\ fa Ph on e : 06 11 88 45 28
z Born on 24/06/2000
\ faAt : krombiyahya@gmail.com
\ faLin kedin : yahya krombi
 Driving License B
PROFESSIONAL EXPERIENCE
Data Scientist/ ML Engineer
At BNP Paribas
z September 2024-September 2025 * Paris, France
• Built a RAG chatbot using LangChain and ChromaDB in Domino Datalab,
capable of analyzing complex PDFs, del

OFFER:
 GROUPE [RGteracars (O) Chttps://gri
AAO DANS VOTRE INTERET cOusiehce (http s://groupecreditagricole.jobs/fr/? idgorigine:
Chttps:/groupecreditagricole.jobs/fr/? an $=)

id_origine=502)
“<= & CREDIT AGRICOLE CIB vy

Accueil (https:/groupecreditagricole.jobs/fr/?id_ origine=502)
Nos offres (https:/groupecreditagricole.jobs/fr/nos-offres/?
id_origine=502)

Data scientist confirmé H/F

cDI

Data scientist confirmé H/F

Modifiée le 22/05/2025

9 Montrouge - France
& IT, Digital et Data

In [26]:
!pip install openai langchain langchain-community
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os

load_dotenv()
def get_rh_chain(cv_text: str, offer_text: str, chat_history: list[str]) -> LLMChain:
    system_template = """
Tu es un recruteur RH bienveillant et professionnel chargé de conduire un entretien.

Voici le CV du candidat :
{cv_text}

Voici l'offre d'emploi :
{offer_text}

Voici l'historique récent de l’entretien :
{chat_history}

Voici la nouvelle entrée du candidat (question ou réponse) :
{user_input}

Ta mission :
- Si c’est une réponse, pose une question RH pertinente pour approfondir.
- Si c’est une question, réponds naturellement, professionnellement et avec empathie.
Sois humain, pertinent et adapté à l’échange.
"""

    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(system_template)
    ])

    llm = ChatOpenAI(model="gpt-4o", temperature=0.4)

    return LLMChain(llm=llm, prompt=prompt)


  Using cached langchain_community-0.3.24-py3-none-any.whl.metadata (2.5 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.9.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.3.24-py3-none-any.whl (2.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.6 MB/s eta 0:00:00a 0:00:01
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached 

In [27]:
chat_history = [
    "RH: Bonjour Yahya, pouvez-vous vous présenter ?",
    "Candidat: Oui, je suis data engineer chez BNP Paribas en alternance.",
    "RH: Pouvez-vous me parler d’un projet marquant ?",
    "Candidat: J’ai développé un chatbot RAG pour analyser des PDFs complexes.",
]

user_input = "Ce projet m’a permis d'apprendre à intégrer LangChain avec ChromaDB."

chain = get_rh_chain(cv_text, offer_text, chat_history)

response = chain.invoke({
    "cv_text": cv_text,
    "offer_text": offer_text,
    "chat_history": "\n".join(chat_history),
    "user_input": user_input
})

print(response)


/tmp/ipykernel_77426/645062676.py:35: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o", temperature=0.4)
/tmp/ipykernel_77426/645062676.py:37: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=llm, prompt=prompt)


{'cv_text': 'KROMBI YAHYA\nDynamic and rigorous engineer seeking a position in the ﬁeld\nof Data/Gen IA.\n\\ faM apMarker : 136 Boulev ard Maxime Gorki, Villejuif\n\\ fa Ph on e : 06 11 88 45 28\nz Born on 24/06/2000\n\\ faAt : krombiyahya@gmail.com\n\\ faLin kedin : yahya krombi\n\x88 Driving License B\nPROFESSIONAL EXPERIENCE\nData Scientist/ ML Engineer\nAt BNP Paribas\nz September 2024-September 2025 * Paris, France\n• Built a RAG chatbot using LangChain and ChromaDB in Domino Datalab,\ncapable of analyzing complex PDFs, delivering real-time, actionable insights\nto inform and support strategic decision-making (LLMs-multimodels, chro-\nmadb, Streamlit, PDF/image-to-data extraction).\n• Enabled multimodal analysis to interpret textual and visual data (Tesser-\nact OCR, Vision Transformers, OpenCV, LangChain tools).\n• Reduced external expenses and streamlined data workﬂows using GenAI\nand LLMs for automated database queries and insight generation (Mistral,\nLangChain agents, SQL, t

In [29]:
print(response["text"])

RH : C'est intéressant, Yahya. Pourriez-vous nous expliquer comment vous avez surmonté les défis techniques liés à l'intégration de LangChain avec ChromaDB ? Quels ont été les principaux obstacles et comment les avez-vous résolus ?
